In [1]:
from tensorflow import keras
import plaidml.keras
plaidml.keras.install_backend()
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers import BatchNormalization, Lambda
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2
# from tensorflow.keras import backend
from tensorflow.python.keras import backend
from keras.preprocessing import image
from keras import backend as K
from keras import optimizers
from keras.layers import Layer
import numpy as np
from glob import glob
import cv2, os, random
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
class LocalResponseNormalization(Layer):
    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1,1), padding="same", pool_mode='avg')
        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom 
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [3]:
# seed 값 설정
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
path='../dataset/cat_and_dog/train'

In [5]:
ROW, COL = 96, 96

dogs, cats = [], []
y_dogs, y_cats = [], []

In [6]:
## Definition to load all our dog images
def load_dogs():
    print('Loading all dog images\n')
    dog_path = os.path.join(path, 'dog*')
    for dog_img in glob(dog_path):
        dog = cv2.imread(dog_img)
        dog = cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY)
        dog = cv2.resize(dog, (ROW, COL))
        dog = image.img_to_array(dog)
        dogs.append(dog)
    print('All dog images loaded')

In [7]:
## Definition to load all our cat images
def load_cats():
    print('Loading all cat images\n')
    cat_path = os.path.join(path, 'cat*')
    for cat_img in glob(cat_path):
        cat = cv2.imread(cat_img)
        cat = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
        cat = cv2.resize(cat, (ROW, COL))
        cat = image.img_to_array(cat)
        cats.append(cat)
    print('All cat images loaded')

In [8]:
load_dogs()

Loading all dog images

All dog images loaded


In [9]:
load_cats()

Loading all cat images

All cat images loaded


In [10]:
classes = ['dog', 'cat']

In [11]:
## in case we want to see if our images was saved correctly in arrays we can use those codes
def show_dogs():
    plt.figure(figsize=(12,8))    
    for i in range(5):
        plt.subplot(1, 5, i+1)
        img = image.array_to_img(random.choice(dogs))
        plt.imshow(img)
        
        plt.axis('off')
        plt.title('Supposed to be a {}'.format(classes[0]))        
    plt.show()

In [12]:
def show_cats():
    plt.figure(figsize=(12,8))
    for i in range(5):
        plt.subplot(1, 5, i+1)
        img = image.array_to_img(random.choice(cats))
        plt.imshow(img)

        plt.axis('off')
        plt.title('Supposed to be a {}'.format(classes[1]))
    plt.show()


In [13]:
## just change the labels for 0 and  1
y_dogs = [1 for item in enumerate(dogs)]
y_cats = [0 for item in enumerate(cats)]

In [14]:
dogs = np.asarray(dogs).astype('float32') / 255
cats = np.asarray(cats).astype('float32') / 255
y_dogs = np.asarray(y_dogs).astype('int32')
y_cats = np.asarray(y_cats).astype('int32')

In [15]:
X = np.concatenate((dogs,cats), axis=0)
y = np.concatenate((y_dogs, y_cats), axis=0)

In [16]:
IMG_CHANNEL = 1
BATCH_SIZE = 128
N_EPOCH = 10
VERBOSE = 2
VALIDAION_SPLIT = .2
OPTIM = 'Adam'
N_CLASSES = len(classes)

In [17]:
## One-Hot Encoding
y = keras.utils.to_categorical(y, N_CLASSES)

In [18]:
input_shape = (ROW, COL, IMG_CHANNEL)

model = Sequential()
model.add(Conv2D(96, (11, 11), strides=4,
                 padding='same', input_shape=input_shape))
model.add(Conv2D(256, (5, 5), activation='relu', padding='same'))
model.add(LocalResponseNormalization(input_shape=model.output_shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(LocalResponseNormalization(input_shape=model.output_shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(N_CLASSES, activation='softmax'))

model.summary()

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 96)        11712     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 256)       614656    
_________________________________________________________________
local_response_normalization (None, 24, 24, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 384)       885120    
_________________________________________________________________
local_response_normalization (None, 11, 11, 384)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 384)         0         
__________

In [19]:
optimizer = optimizers.SGD(lr=0.01, decay=5e-5, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=OPTIM, metrics=['accuracy'])

In [23]:
modelpath = 'model/dog.h5'
checkpoint = ModelCheckpoint(filepath=modelpath)

In [24]:
model.fit(X, y, batch_size=BATCH_SIZE, epochs=N_EPOCH, 
          validation_split=VALIDAION_SPLIT,
          verbose=VERBOSE, callbacks=[checkpoint])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 68s - loss: 0.6633 - acc: 0.6243 - val_loss: 0.8017 - val_acc: 0.0000e+00
Epoch 2/10
 - 68s - loss: 0.6605 - acc: 0.6259 - val_loss: 0.8288 - val_acc: 0.1038
Epoch 3/10
 - 68s - loss: 0.6509 - acc: 0.6371 - val_loss: 0.9845 - val_acc: 0.0672
Epoch 4/10
 - 68s - loss: 0.6365 - acc: 0.6470 - val_loss: 0.8359 - val_acc: 0.3332
Epoch 5/10
 - 67s - loss: 0.6248 - acc: 0.6588 - val_loss: 1.0318 - val_acc: 0.2306
Epoch 6/10
 - 66s - loss: 0.6094 - acc: 0.6710 - val_loss: 1.0038 - val_acc: 0.1790
Epoch 7/10
 - 68s - loss: 0.5973 - acc: 0.6863 - val_loss: 0.8069 - val_acc: 0.5050
Epoch 8/10
 - 68s - loss: 0.5849 - acc: 0.6932 - val_loss: 0.7540 - val_acc: 0.5266
Epoch 9/10
 - 68s - loss: 0.5982 - acc: 0.6980 - val_loss: 0.7492 - val_acc: 0.5220
Epoch 10/10
 - 68s - loss: 0.5643 - acc: 0.7121 - val_loss: 0.9758 - val_acc: 0.3578


In [25]:
scores = model.evaluate(X, y, verbose=2)
print('MODEL ACCURACY: %.5f' % scores[1])

MODEL ACCURACY: 0.64924
